<a href="https://colab.research.google.com/github/laribar/bitcoinprediction/blob/main/modelo_yfinance_geral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para baixar dados e calcular métricas técnicas
def baixar_dados(acao, intervalo="1d", periodo="2y"):
    try:
        df = yf.download(acao, period=periodo, interval=intervalo, progress=False, auto_adjust=False)
        if df.empty or df.isnull().values.any():
            logging.error(f"Erro: Dados faltantes ou inválidos para {acao}.")
            return None

        preco_coluna = "Adj Close" if "Adj Close" in df.columns else "Close"

        if preco_coluna not in df.columns:
            logging.error(f"Erro ao processar {acao}: Nenhuma coluna de fechamento encontrada nos dados.")
            return None

        # Remove NaNs
        df.dropna(inplace=True)

        # Calcula a média móvel simples (SMA) e desvio padrão com shift para evitar forward-looking bias
        df["SMA"] = df[preco_coluna].rolling(window=20).mean().shift(1)
        df["STD"] = df[preco_coluna].rolling(window=20).std().shift(1)

        # Calcula as bandas superior e inferior
        df["Upper_Band"] = df["SMA"] + (df["STD"] * 2)
        df["Lower_Band"] = df["SMA"] - (df["STD"] * 2)

        # Calcula suporte e resistência com shift
        df["Support"] = df["Low"].rolling(window=20).min().shift(1)
        df["Resistance"] = df["High"].rolling(window=20).max().shift(1)

        # Calcula LTAs e LTBs com lookback dinâmico baseado na volatilidade
        lookback = max(10, int(df["STD"].mean() * 5))  # Ajuste baseado na volatilidade
        df["LTA"] = df["High"].rolling(window=lookback).max().shift(1)
        df["LTB"] = df["Low"].rolling(window=lookback).min().shift(1)

        # Remove colunas auxiliares que não serão usadas no modelo
        df = df.drop(columns=["STD"])

        return df

    except Exception as e:
        logging.error(f"Erro ao baixar dados para {acao}: {e}", exc_info=True)
        return None

# Função para treinar o modelo e prever preços futuros
def prever_precos(df, acao, intervalo):
    try:
        # Definir target como o preço do dia seguinte
        df["Target"] = df["Close"].shift(-1)
        df.dropna(inplace=True)  # Remove a última linha com target NaN

        # Selecionar features e target
        X = df[['SMA', 'Upper_Band', 'Lower_Band', 'Support', 'Resistance', 'LTA', 'LTB']]
        y = df['Target'].values.ravel()  # Converter y para array 1D

        # Definir cutoff_date como 98% dos dados
        cutoff_index = int(0.98 * len(df))  # Índice numérico para corte
        X_train, X_test = X.iloc[:cutoff_index], X.iloc[cutoff_index:]
        y_train, y_test = y[:cutoff_index], y[cutoff_index:]

        # Verificar se há dados suficientes para treinamento e teste
        if len(X_train) == 0 or len(X_test) == 0:
            logging.error(f"Erro: Dados insuficientes para treinamento/teste em {acao} no intervalo {intervalo}")
            return None

        # Padronizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Ajuste de hiperparâmetros com GridSearchCV e TimeSeriesSplit
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
        model = RandomForestRegressor(random_state=42)
        tss = TimeSeriesSplit(n_splits=3)
        grid_search = GridSearchCV(model, param_grid, cv=tss, scoring='neg_mean_squared_error')
        grid_search.fit(X_train_scaled, y_train)

        best_model = grid_search.best_estimator_

        # Criar datas futuras
        future_dates = [df.index[-1] + pd.Timedelta(days=i) for i in range(1, 6)]

        # Criar features futuras com base em tendências
        future_features = pd.DataFrame({
            'SMA': [df['SMA'].iloc[-1] + i * 0.1 for i in range(5)],
            'Upper_Band': [df['Upper_Band'].iloc[-1] + i * 0.1 for i in range(5)],
            'Lower_Band': [df['Lower_Band'].iloc[-1] + i * 0.1 for i in range(5)],
            'Support': [df['Support'].iloc[-1] + i * 0.1 for i in range(5)],
            'Resistance': [df['Resistance'].iloc[-1] + i * 0.1 for i in range(5)],
            'LTA': [df['LTA'].iloc[-1] + i * 0.1 for i in range(5)],
            'LTB': [df['LTB'].iloc[-1] + i * 0.1 for i in range(5)]
        }, index=future_dates)

        # Transformar features futuras usando o scaler ajustado
        future_features_scaled = scaler.transform(future_features)

        # Fazer previsões
        future_predictions = best_model.predict(future_features_scaled)
        return future_predictions

    except Exception as e:
        logging.error(f"Erro ao prever preços para {acao}: {e}", exc_info=True)
        return None

# Função para gerar alertas de compra/venda
def gerar_alertas(df, previsoes, acao, intervalo):
    try:
        # Garantir que last_price seja um valor escalar
        last_price = df['Close'].iloc[-1]
        if isinstance(last_price, pd.Series):
            last_price = last_price.item()  # Converte para um valor escalar

        if previsoes is not None and len(previsoes) > 0:
            avg_future_price = np.mean(previsoes)

            # Garantir que todas as métricas sejam valores escalares
            sma = df['SMA'].iloc[-1].item() if isinstance(df['SMA'].iloc[-1], pd.Series) else df['SMA'].iloc[-1]
            upper_band = df['Upper_Band'].iloc[-1].item() if isinstance(df['Upper_Band'].iloc[-1], pd.Series) else df['Upper_Band'].iloc[-1]
            lower_band = df['Lower_Band'].iloc[-1].item() if isinstance(df['Lower_Band'].iloc[-1], pd.Series) else df['Lower_Band'].iloc[-1]
            support = df['Support'].iloc[-1].item() if isinstance(df['Support'].iloc[-1], pd.Series) else df['Support'].iloc[-1]
            resistance = df['Resistance'].iloc[-1].item() if isinstance(df['Resistance'].iloc[-1], pd.Series) else df['Resistance'].iloc[-1]
            lta = df['LTA'].iloc[-1].item() if isinstance(df['LTA'].iloc[-1], pd.Series) else df['LTA'].iloc[-1]
            ltb = df['LTB'].iloc[-1].item() if isinstance(df['LTB'].iloc[-1], pd.Series) else df['LTB'].iloc[-1]

            # Definir ordens de compra e venda
            ordem_compra = max(support, lower_band)  # Ordem de compra: máximo entre suporte e banda inferior
            ordem_venda = min(resistance, upper_band)  # Ordem de venda: mínimo entre resistência e banda superior

            # Gerar alerta e justificativa
            if avg_future_price > last_price * 1.02:  # Tendência de alta
                alerta = '🚀 Compra sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% maior que o preço atual ({last_price:.2f}). "
                    f"Além disso, a SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, e a LTA em {lta:.2f}. "
                    f"Isso indica uma tendência de alta."
                )
            elif avg_future_price < last_price * 0.98:  # Tendência de baixa
                alerta = '⚠️ Venda sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% menor que o preço atual ({last_price:.2f}). "
                    f"Além disso, a SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, e a LTB em {ltb:.2f}. "
                    f"Isso indica uma tendência de baixa."
                )
            else:
                alerta = '🔍 Acompanhar'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) está dentro de 2% do preço atual ({last_price:.2f}). "
                    f"A SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, a LTA em {lta:.2f}, e a LTB em {ltb:.2f}. "
                    f"Isso sugere que o mercado está estável no momento."
                )
        else:
            alerta = '❌ Sem previsões disponíveis'
            justificativa = 'Não há previsões disponíveis para esta ação.'
            ordem_compra = None
            ordem_venda = None

        # Exibir resultados
        print(f"Stock: {acao} | Intervalo: {intervalo}")
        print(f"Last price: {last_price:.2f}")
        print(f"Predictions: {previsoes}")
        print(f"Alert: {alerta}")
        print(f"Ordem de Compra: {ordem_compra:.2f}" if ordem_compra else "Ordem de Compra: N/A")
        print(f"Ordem de Venda: {ordem_venda:.2f}" if ordem_venda else "Ordem de Venda: N/A")
        print(f"Justificativa: {justificativa}")
        print("-" * 80)

    except Exception as e:
        logging.error(f"Erro ao gerar alertas para {acao}: {e}", exc_info=True)

# Função para obter o Top 10 da NASDAQ por capitalização de mercado
def obter_top_10_nasdaq():
    """
    Obtém o Top 10 da NASDAQ por capitalização de mercado.
    """
    # Baixar a lista de tickers da NASDAQ
    nasdaq_tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
    top_10 = nasdaq_tickers.head(10)['Ticker'].tolist()
    return top_10

# Processar ações
acoes = obter_top_10_nasdaq()
for acao in acoes:
    df = baixar_dados(acao)
    if df is not None:
        previsoes = prever_precos(df, acao, "1d")
        gerar_alertas(df, previsoes, acao, "1d")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: ADBE | Intervalo: 1d
Last price: 387.89
Predictions: [418.38059456 418.3857943  418.38099419 418.38099419 418.38099419]
Alert: 🚀 Compra sugerida
Ordem de Compra: 387.51
Ordem de Venda: 463.61
Justificativa: A média das previsões futuras (418.38) é 2% maior que o preço atual (387.89). Além disso, a SMA está em 433.19, a banda superior em 478.87, e a banda inferior em 387.51. O suporte está em 374.50, a resistência em 463.61, e a LTA em 557.90. Isso indica uma tendência de alta.
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: AMD | Intervalo: 1d
Last price: 106.23
Predictions: [111.85259796 111.95829788 111.60279785 111.60279785 111.33949776]
Alert: 🚀 Compra sugerida
Ordem de Compra: 94.73
Ordem de Venda: 113.55
Justificativa: A média das previsões futuras (111.67) é 2% maior que o preço atual (106.23). Além disso, a SMA está em 102.85, a banda superior em 113.55, e a banda inferior em 92.15. O suporte está em 94.73, a resistência em 116.55, e a LTA em 120.44. Isso indica uma tendência de alta.
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: AMGN | Intervalo: 1d
Last price: 315.75
Predictions: [288.83095893 288.83095893 288.83095893 288.83095893 289.05178962]
Alert: ⚠️ Venda sugerida
Ordem de Compra: 295.81
Ordem de Venda: 328.59
Justificativa: A média das previsões futuras (288.88) é 2% menor que o preço atual (315.75). Além disso, a SMA está em 312.20, a banda superior em 328.59, e a banda inferior em 295.81. O suporte está em 291.80, a resistência em 335.88, e a LTB em 279.31. Isso indica uma tendência de baixa.
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: ADI | Intervalo: 1d
Last price: 209.97
Predictions: [218.486783   218.94878312 218.68478318 218.68478318 218.8769835 ]
Alert: 🚀 Compra sugerida
Ordem de Compra: 202.59
Ordem de Venda: 247.10
Justificativa: A média das previsões futuras (218.74) é 2% maior que o preço atual (209.97). Além disso, a SMA está em 223.67, a banda superior em 248.03, e a banda inferior em 199.30. O suporte está em 202.59, a resistência em 247.10, e a LTA em 247.10. Isso indica uma tendência de alta.
--------------------------------------------------------------------------------
Stock: ANSS | Intervalo: 1d
Last price: 323.04
Predictions: [329.52465361 329.53816756 329.50111609 329.50111609 329.50111609]
Alert: 🚀 Compra sugerida
Ordem de Compra: 314.15
Ordem de Venda: 340.26
Justificativa: A média das previsões futuras (329.51) é 2% maior que o preço atual (323.04). Além disso, a SMA está em 327.21, a banda superior em 340.26, e a banda inferior em 314.15. O suporte está em 314.11, a resistência em 345.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
